# GAN - PyTorch version

In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [4]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])]) 

In [5]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./MNIST_data/',
                                   train=True,
                                   transform=transform,
                                   download=True)


0it [00:00, ?it/s]

100%|███████████████████████████████████████████████████████████████████▊| 9879552/9912422 [00:33<00:00, 372201.66it/s]

Extracting ./data/MNIST\raw\train-images-idx3-ubyte.gz to ./data/MNIST\raw



0it [00:00, ?it/s]


 85%|██████████████████████████████████████████████████████████████           | 24576/28881 [00:01<00:00, 44135.80it/s]
32768it [00:01, 17451.16it/s]                                                                                          

0it [00:00, ?it/s]

...


 78%|████████████████████████████████████████████████████▋               | 1277952/1648877 [00:05<00:01, 347472.60it/s]

0it [00:00, ?it/s]

Extracting ./data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/MNIST\raw




8192it [00:00, 8353.07it/s]                                                                                            

Extracting ./data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/MNIST\raw
Processing...
Done!



9920512it [00:50, 372201.66it/s]                                                                                       
1654784it [00:23, 347472.60it/s]                                                                                       

In [6]:
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True) 

In [7]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())
D

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): LeakyReLU(negative_slope=0.2)
  (4): Linear(in_features=256, out_features=1, bias=True)
  (5): Sigmoid()
)

In [8]:
# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())
G

Sequential(
  (0): Linear(in_features=64, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=784, bias=True)
  (5): Tanh()
)

In [9]:
# Device setting
D = D.to(device)
G = G.to(device) 

In [10]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002) 

In [11]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad() 

In [12]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

Epoch [0/200], Step [200/600], d_loss: 0.0387, g_loss: 4.4964, D(x): 0.99, D(G(z)): 0.03
Epoch [0/200], Step [400/600], d_loss: 0.7588, g_loss: 4.2267, D(x): 0.76, D(G(z)): 0.14
Epoch [0/200], Step [600/600], d_loss: 0.1422, g_loss: 4.8025, D(x): 0.94, D(G(z)): 0.07
Epoch [1/200], Step [200/600], d_loss: 0.1057, g_loss: 3.9436, D(x): 0.97, D(G(z)): 0.07
Epoch [1/200], Step [400/600], d_loss: 0.2951, g_loss: 3.2131, D(x): 0.92, D(G(z)): 0.13
Epoch [1/200], Step [600/600], d_loss: 0.1440, g_loss: 4.6858, D(x): 0.95, D(G(z)): 0.06
Epoch [2/200], Step [200/600], d_loss: 0.3493, g_loss: 3.5372, D(x): 0.90, D(G(z)): 0.10
Epoch [2/200], Step [400/600], d_loss: 0.2674, g_loss: 3.9863, D(x): 0.91, D(G(z)): 0.06
Epoch [2/200], Step [600/600], d_loss: 0.3307, g_loss: 3.8158, D(x): 0.93, D(G(z)): 0.18
Epoch [3/200], Step [200/600], d_loss: 0.3801, g_loss: 3.4144, D(x): 0.92, D(G(z)): 0.19
Epoch [3/200], Step [400/600], d_loss: 0.1559, g_loss: 3.7532, D(x): 0.94, D(G(z)): 0.06
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.2383, g_loss: 4.2820, D(x): 0.94, D(G(z)): 0.10
Epoch [31/200], Step [200/600], d_loss: 0.3337, g_loss: 4.9028, D(x): 0.87, D(G(z)): 0.06
Epoch [31/200], Step [400/600], d_loss: 0.2378, g_loss: 2.9350, D(x): 0.95, D(G(z)): 0.12
Epoch [31/200], Step [600/600], d_loss: 0.3706, g_loss: 3.7365, D(x): 0.94, D(G(z)): 0.19
Epoch [32/200], Step [200/600], d_loss: 0.4023, g_loss: 4.6450, D(x): 0.86, D(G(z)): 0.08
Epoch [32/200], Step [400/600], d_loss: 0.3327, g_loss: 3.6184, D(x): 0.93, D(G(z)): 0.16
Epoch [32/200], Step [600/600], d_loss: 0.2996, g_loss: 3.6713, D(x): 0.88, D(G(z)): 0.05
Epoch [33/200], Step [200/600], d_loss: 0.5807, g_loss: 2.6442, D(x): 0.82, D(G(z)): 0.13
Epoch [33/200], Step [400/600], d_loss: 0.6333, g_loss: 2.8383, D(x): 0.86, D(G(z)): 0.23
Epoch [33/200], Step [600/600], d_loss: 0.3824, g_loss: 2.7578, D(x): 0.90, D(G(z)): 0.16
Epoch [34/200], Step [200/600], d_loss: 0.3566, g_loss: 3.7827, D(x): 0.92, D(G(z)): 0.15
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.7754, g_loss: 2.8710, D(x): 0.71, D(G(z)): 0.14
Epoch [61/200], Step [600/600], d_loss: 0.6307, g_loss: 2.3899, D(x): 0.79, D(G(z)): 0.20
Epoch [62/200], Step [200/600], d_loss: 0.4936, g_loss: 2.2813, D(x): 0.86, D(G(z)): 0.21
Epoch [62/200], Step [400/600], d_loss: 0.8599, g_loss: 2.2808, D(x): 0.70, D(G(z)): 0.22
Epoch [62/200], Step [600/600], d_loss: 0.6597, g_loss: 2.1527, D(x): 0.76, D(G(z)): 0.17
Epoch [63/200], Step [200/600], d_loss: 0.7586, g_loss: 1.7833, D(x): 0.80, D(G(z)): 0.30
Epoch [63/200], Step [400/600], d_loss: 0.6213, g_loss: 2.7678, D(x): 0.81, D(G(z)): 0.20
Epoch [63/200], Step [600/600], d_loss: 0.7673, g_loss: 2.0924, D(x): 0.80, D(G(z)): 0.28
Epoch [64/200], Step [200/600], d_loss: 0.6921, g_loss: 2.6730, D(x): 0.69, D(G(z)): 0.14
Epoch [64/200], Step [400/600], d_loss: 0.7044, g_loss: 1.8864, D(x): 0.84, D(G(z)): 0.31
Epoch [64/200], Step [600/600], d_loss: 0.6827, g_loss: 2.4694, D(x): 0.71, D(G(z)): 0.15
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.9320, g_loss: 1.8236, D(x): 0.73, D(G(z)): 0.33
Epoch [92/200], Step [400/600], d_loss: 0.8031, g_loss: 2.0995, D(x): 0.74, D(G(z)): 0.26
Epoch [92/200], Step [600/600], d_loss: 0.6949, g_loss: 1.7164, D(x): 0.73, D(G(z)): 0.21
Epoch [93/200], Step [200/600], d_loss: 1.0894, g_loss: 1.9526, D(x): 0.60, D(G(z)): 0.26
Epoch [93/200], Step [400/600], d_loss: 0.7248, g_loss: 2.1129, D(x): 0.74, D(G(z)): 0.19
Epoch [93/200], Step [600/600], d_loss: 0.7956, g_loss: 2.4027, D(x): 0.64, D(G(z)): 0.15
Epoch [94/200], Step [200/600], d_loss: 0.8278, g_loss: 1.7310, D(x): 0.69, D(G(z)): 0.22
Epoch [94/200], Step [400/600], d_loss: 0.7678, g_loss: 1.6488, D(x): 0.70, D(G(z)): 0.21
Epoch [94/200], Step [600/600], d_loss: 0.9339, g_loss: 1.5563, D(x): 0.81, D(G(z)): 0.39
Epoch [95/200], Step [200/600], d_loss: 0.7856, g_loss: 2.0009, D(x): 0.83, D(G(z)): 0.33
Epoch [95/200], Step [400/600], d_loss: 0.8364, g_loss: 2.4504, D(x): 0.71, D(G(z)): 0.22
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8261, g_loss: 1.5699, D(x): 0.71, D(G(z)): 0.30
Epoch [122/200], Step [600/600], d_loss: 0.8608, g_loss: 1.9797, D(x): 0.62, D(G(z)): 0.18
Epoch [123/200], Step [200/600], d_loss: 0.8244, g_loss: 1.6487, D(x): 0.73, D(G(z)): 0.28
Epoch [123/200], Step [400/600], d_loss: 0.6528, g_loss: 1.4506, D(x): 0.77, D(G(z)): 0.24
Epoch [123/200], Step [600/600], d_loss: 1.0392, g_loss: 1.5213, D(x): 0.65, D(G(z)): 0.31
Epoch [124/200], Step [200/600], d_loss: 0.9445, g_loss: 1.3160, D(x): 0.73, D(G(z)): 0.34
Epoch [124/200], Step [400/600], d_loss: 1.0165, g_loss: 1.4271, D(x): 0.67, D(G(z)): 0.30
Epoch [124/200], Step [600/600], d_loss: 0.8504, g_loss: 1.3124, D(x): 0.73, D(G(z)): 0.30
Epoch [125/200], Step [200/600], d_loss: 0.7421, g_loss: 1.5789, D(x): 0.72, D(G(z)): 0.25
Epoch [125/200], Step [400/600], d_loss: 0.8707, g_loss: 1.7348, D(x): 0.74, D(G(z)): 0.34
Epoch [125/200], Step [600/600], d_loss: 1.0286, g_loss: 1.5519, D(x): 0.63, D(G(z)): 0.29

Epoch [152/200], Step [600/600], d_loss: 0.9453, g_loss: 1.2143, D(x): 0.76, D(G(z)): 0.37
Epoch [153/200], Step [200/600], d_loss: 0.9921, g_loss: 1.6186, D(x): 0.72, D(G(z)): 0.35
Epoch [153/200], Step [400/600], d_loss: 1.0014, g_loss: 1.4269, D(x): 0.70, D(G(z)): 0.35
Epoch [153/200], Step [600/600], d_loss: 0.8259, g_loss: 1.5639, D(x): 0.72, D(G(z)): 0.29
Epoch [154/200], Step [200/600], d_loss: 0.9269, g_loss: 1.4939, D(x): 0.76, D(G(z)): 0.37
Epoch [154/200], Step [400/600], d_loss: 1.0780, g_loss: 1.6757, D(x): 0.62, D(G(z)): 0.31
Epoch [154/200], Step [600/600], d_loss: 0.8294, g_loss: 1.4396, D(x): 0.70, D(G(z)): 0.28
Epoch [155/200], Step [200/600], d_loss: 1.1062, g_loss: 1.7145, D(x): 0.65, D(G(z)): 0.38
Epoch [155/200], Step [400/600], d_loss: 0.8468, g_loss: 1.6180, D(x): 0.67, D(G(z)): 0.27
Epoch [155/200], Step [600/600], d_loss: 1.0616, g_loss: 1.5408, D(x): 0.62, D(G(z)): 0.32
Epoch [156/200], Step [200/600], d_loss: 0.8612, g_loss: 1.5601, D(x): 0.75, D(G(z)): 0.34

Epoch [183/200], Step [200/600], d_loss: 1.0192, g_loss: 1.3456, D(x): 0.64, D(G(z)): 0.29
Epoch [183/200], Step [400/600], d_loss: 0.9956, g_loss: 1.3738, D(x): 0.68, D(G(z)): 0.34
Epoch [183/200], Step [600/600], d_loss: 1.1413, g_loss: 1.4322, D(x): 0.60, D(G(z)): 0.33
Epoch [184/200], Step [200/600], d_loss: 0.9897, g_loss: 1.4320, D(x): 0.68, D(G(z)): 0.35
Epoch [184/200], Step [400/600], d_loss: 0.8402, g_loss: 1.8036, D(x): 0.76, D(G(z)): 0.33
Epoch [184/200], Step [600/600], d_loss: 0.9482, g_loss: 1.7058, D(x): 0.68, D(G(z)): 0.32
Epoch [185/200], Step [200/600], d_loss: 0.9956, g_loss: 1.6483, D(x): 0.70, D(G(z)): 0.35
Epoch [185/200], Step [400/600], d_loss: 1.2101, g_loss: 1.2622, D(x): 0.60, D(G(z)): 0.35
Epoch [185/200], Step [600/600], d_loss: 0.8214, g_loss: 1.7451, D(x): 0.70, D(G(z)): 0.27
Epoch [186/200], Step [200/600], d_loss: 0.8823, g_loss: 1.4678, D(x): 0.73, D(G(z)): 0.34
Epoch [186/200], Step [400/600], d_loss: 1.0793, g_loss: 1.3707, D(x): 0.69, D(G(z)): 0.41

In [13]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'model/G.ckpt')
torch.save(D.state_dict(), 'model/D.ckpt')